works with https://github.com/mocnik-science/osm-python-tools/ library

categories are presented for example here https://github.com/GIScience/openpoiservice/blob/master/categories_docker.yml

In [19]:
import pandas as pd
import os
import time
from OSMPythonTools.overpass import overpassQueryBuilder
from OSMPythonTools.overpass import Overpass
overpass = Overpass(waitBetweenQueries = 50)

### importing data

In [27]:
# table of items to request
request_list = pd.read_excel('./input_params.xlsx', sheet_name = 'overpass_example')

# lists of items without missing values
for item_type in list(request_list.columns):
    list_of_items = request_list[item_type].dropna().to_list()
    print(list_of_items)

['hotel', 'motel']
['library', 'museum', 'bank', 'hospital', 'cafe', 'fast_food', 'pub', 'restaurant']
['park', 'garden']
['shoes', 'alcohol', 'bakery', 'cheese', 'tobacco']


In [28]:
list(request_list.columns)

['tourism', 'amenity', 'leisure', 'shop']

In [ ]:
# create dir for output files
os.makedirs("output", exist_ok=True)

### request

In [22]:
custom_bbox = [48.1, 16.3, 48.3, 16.5]

In [23]:
def poi_request(item_type, search_query):
    # query built
    query = overpassQueryBuilder(bbox=custom_bbox, elementType='node', 
                                     selector='"{}"="{}"'.format(str(item_type), str(search_query)), out='body')
    result = overpass.query(query, timeout=100000)

    # result in json
    result = result.toJSON()['elements']
    
    # separate tags
    for row in result:
        row.update(row['tags'])

    df = pd.DataFrame(result)
    return df

In [24]:
for item_type in list(request_list.columns):
    # list of queries from column
    list_of_items = request_list[item_type].dropna().to_list()
    
    # request for items in item category
    for search_query in list_of_items:
        df = poi_request(item_type, search_query)
        time.sleep(100)

        df.to_csv("./output/{}.csv".format(str(search_query)))

### check result

In [25]:
# lists of items without missing values
for item_type in list(request_list.columns):
    list_of_items = request_list[item_type].dropna().to_list()
    print(list_of_items)

['hotel', 'motel']
['library', 'museum', 'bank', 'hospital']
['park', 'garden']
['cafe', 'fast_food', 'pub', 'restaurant']


In [26]:
test = pd.read_csv('./output/hotel.csv', index_col=0)
test.head(10)

,type,id,lat,lon,tags,addr:city,addr:country,addr:housenumber,addr:postcode,addr:street,...,source:imagery,contact:email,cuisine,beds,payment:american_express,brand:wikipedia,capacity,name:de,contact:fax,at_bev:addr_date
0,node,122097286,48.226427,16.360097,"{'addr:city': 'Wien', 'addr:country': 'AT', 'a...",Wien,AT,5,1090.0,Althanstraße,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,144604712,48.166765,16.345039,"{'addr:city': 'Wien', 'addr:country': 'AT', 'a...",Wien,AT,5,1100.0,Hertha-Firnberg-Straße,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,218118487,48.163911,16.323244,"{'addr:city': 'Wien', 'addr:country': 'AT', 'a...",Wien,AT,33,1120.0,Hoffingergasse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,270064680,48.191965,16.339198,"{'addr:city': 'Wien', 'addr:country': 'AT', 'a...",Wien,AT,23,1060.0,Wallgasse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,276465063,48.217065,16.389498,"{'addr:city': 'Wien', 'addr:country': 'AT', 'a...",Wien,AT,72,1020.0,Praterstraße,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,node,279452020,48.189361,16.372282,"{'name': 'Hotel beim Theresianum', 'tourism': ...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,node,297277443,48.194980,16.369402,"{'addr:city': 'Wien', 'addr:country': 'AT', 'a...",Wien,AT,12,1040.0,Favoritenstraße,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,node,302737734,48.196450,16.339925,"{'addr:city': 'Wien', 'addr:country': 'AT', 'a...",Wien,AT,4,1070.0,Neubaugürtel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,node,305583433,48.196875,16.364544,"{'designation': '3 star', 'name': 'Carlton Ope...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,node,309293356,48.202352,16.357394,"{'addr:city': 'Wien', 'addr:country': 'AT', 'a...",Wien,AT,7,1070.0,Karl-Schweighofer-Gasse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
